In [ ]:
import os

HUGGINGFACEHUB_API_TOKEN = ''
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPER_API_KEY"] = ''
os.environ['OPENAI_API_KEY'] = ''

Retrieval, including loading vectordb

In [ ]:
import openai
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
openai.api_key = ''

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

# Load from disk
vectordb = Chroma(collection_name='v_db', persist_directory='../docs/chroma14', embedding_function=embedding)

C:\Users\ikhba\AppData\Local\Temp\ipykernel_3176\520700564.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(collection_name='v_db', persist_directory='../docs/chroma14', embedding_function=embedding)
⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


Find which class prompt belongs to

In [5]:
from transformers import AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("bayartsogt/mongolian-gpt2")
tokenizer.pad_token = tokenizer.eos_token

generator = pipeline("text-generation", model="Ikhee10/khasbank_three_classifier_v13", tokenizer = tokenizer, device=0, num_beams=5)

c:\Users\ikhba\Documents\ReAct\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
import re

def process_generated_text(text):
    # Example regex to extract data in the format you've specified
    matches = re.findall(r'<(.*?)>', text)
    ans = matches[1] if len(matches) > 0 else ''
    if ' ' in ans:
        ans = ans.split(' ')[0].strip()
    return ans
    
def generate_and_process(prompt):
    result = generator(prompt, max_length=32, pad_token_id=tokenizer.eos_token_id)
    generated_text = result[0]['generated_text']

    return process_generated_text(generated_text)

In [ ]:
# !pip install google-cloud-translate

In [ ]:
# !pip install --upgrade google-api-python-client
# !pip install --upgrade google-cloud

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.6 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/12.6 MB 2.8 MB/s eta 0:00:05
   ----- ---------------------------------- 1.8/12.6 MB 2.7 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.6 MB 2.9 MB/s eta 0:00:04
   --------- ------------------------------ 2.9/12.6 MB 2.8 MB/s eta 0:00:04
   ---------- ----------------------------- 3.1/12.6 MB 2.6 MB/s eta 0:00:04
   ----------- ---------------------------- 3.7/12.6 MB 2.5 MB/s eta 0:00:04
   ------------ --------------------------- 3.9/12.6 MB 2.4 MB/s eta 0:00:04
   -------------- ------------------------- 4.5/12.6 MB 2.3 MB/s eta 0:00:04
   --------------- ------------------------ 4.7/12.6 MB 2.3 MB/s eta 0:00:04
   --------------- ------------------------ 5.0/12.6 MB 2.2 MB/s eta 0:00:04
   ----------

In [ ]:
from google.cloud import translate_v2 as translate

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'gen-lang-client-0314852010-3b65ae414b7f.json'

translate_client = translate.Client()

In [7]:
def chroma_retrieval_tool(query): 
    # Retrieve documents relevant to the query 
    query_mn = translate_client.translate(query, target_language="mn")["translatedText"]
    clas = generate_and_process('<s> bb: ' + query_mn)
    docs = vectordb.similarity_search(query_mn, k=5, filter={"type": clas}) 
    # Combine retrieved documents into a single string for context 
    context = "\n".join([doc.page_content for doc in docs]) 
    return context

LLaMA generation

In [8]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "meta-llama/Meta-Llama-3-70B-Instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.1,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

Agents

In [9]:
from langchain.agents import load_tools, initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

In [10]:
template = """Доорх асуултанд хариулаарай. Хариулахад дараах мэдээллийг ашиглана уу. Хэрэв та хариултаа мэдэхгүй бол мэдэхгүй гэж хэлээрэй, хариулт зохиох гэж бүү оролдоорой. Хамгийн ихдээ гурван өгүүлбэр ашигла. Хариултаа аль болох товчхон бичээрэй. Хариултын төгсгөлд "Асуусанд баярлалаа!" гэж үргэлж хэлээрэй.
    {context}
    Асуулт: {query}
    Хариулт: Хариулахад алхам алхмаар бодож үзье."""

# Define the REACT_DOCSTORE tool 
react_docstore_tool = Tool( 
    name="react-docstore", 
    # description="A zero shot agent that does a reasoning step before acting. This agent has access to a document store that allows it to look up relevant information to answering the question.", 
    description="Мэдээлэл авахдаа эхлээд боддог дараа нь үйлдэл хийдэг агент. Энэ агент нь асуултад хариулахын тулд холбогдох мэдээллийг хайх боломжтой мэдээллийн санг ашиглах боломжтой.",
    func=chroma_retrieval_tool 
)
tools = load_tools(["google-serper", "llm-math"], llm=llm)
# tools = load_tools(["google-serper"], llm=llm)
tools.append(react_docstore_tool)

In [11]:
# Initialize conversation memory
memory = ConversationBufferMemory()

agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    memory=memory, 
    verbose=True, 
    prompt=template
)

C:\Users\ikhba\AppData\Local\Temp\ipykernel_3176\4260697762.py:4: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


In [16]:
res = agent.run("Хасбанкны хамгийн өндөр хүүтэй хадгаламж ямар хадгаламж вэ?")
res = translate_client.translate(res, target_language="mn")["translatedText"]
print(res)



> Entering new AgentExecutor chain...
 I need to find information about Hasbank's highest interest rate savings account.
Action: google_serper
Action Input: "Hasbank highest interest rate savings account"
Observation: Bask Bank is known for its high-yielding Bask Interest Savings account. It's also known for a unique account, the Bask Mileage Savings account, where you can ... Missing: Hasbank | Show results with:Hasbank. Members without either Direct Deposit or Qualifying Deposits, during the 30-Day Evaluation Period will earn 1.20% APY on savings balances ( ... Missing: Hasbank | Show results with:Hasbank. The Ally Bank Savings Account is a high-yield savings option with no minimum deposit required to open and zero monthly fees. At 3.85% APY, this ... Missing: Hasbank | Show results with:Hasbank. The Openbank High Yield Savings has a high savings account interest rate of 5.00% (vary depending on location) APY. It doesn't charge any ... Missing: Hasbank | Show results with:Hasbank. 

In [20]:
agent.run("Улаанбаатарт үеэр болоод байна. Ямар асуудлаас болж байна вэ? Жагсааж бич.")



> Entering new AgentExecutor chain...
 I need to find out what kind of issues are causing air pollution in Ulaanbaatar.
Action: google_serper
Action Input: "air pollution in Ulaanbaatar"
Observation: Ulaanbaatar – home to half of Mongolia's three million population - is one of the most polluted capitals in the world. On the coldest days of the year, daily average of PM2. 5 pollution levels reach 687 micrograms per cubic meter — 27 times the level WHO recommends as safe.
Thought: I need to find out what are the main causes of air pollution in Ulaanbaatar.
Action: google_serper
Action Input: "main causes of air pollution in Ulaanbaatar"
Observation: “The coal used in the stoves is a primary cause of Ulaanbaatar's air pollution.”
Thought: I need to find out more about the coal used in stoves in Ulaanbaatar.
Action: google_serper
Action Input: "coal used in stoves in Ulaanbaatar"
Observation: Households with energy efficient (EE) stoves fueled less often, but with more coal per fueling, 

'The main causes of air pollution in Ulaanbaatar are the coal used in stoves, coal combustion in gers, and burning coal for domestic heating.'

In [22]:
agent.run("Хас банкны доллар, төгрөг валютын ханш хэд байна вэ?")



> Entering new AgentExecutor chain...
 I need to find the current exchange rate of the USD to MNT.
Action: google_serper
Action Input: "current exchange rate USD to MNT"
Observation: Latest Currency Exchange Rates: 1 US Dollar = 3398 Mongolian Tugrik.
Thought: I have the current exchange rate.
Final Answer: 1 USD is equal to 3398 MNT.

> Finished chain.


'1 USD is equal to 3398 MNT.'

In [24]:
agent.run("Хасбанкны хамгийн өндөр хүүтэй хадгаламж ямар хадгаламж вэ?")



> Entering new AgentExecutor chain...
 I need to find information about Hasbank's highest interest rate savings account.
Action: google_serper
Action Input: "Hasbank highest interest rate savings account"
Observation: Bask Bank is known for its high-yielding Bask Interest Savings account. It's also known for a unique account, the Bask Mileage Savings account, where you can ... Missing: Hasbank | Show results with:Hasbank. Members without either Direct Deposit or Qualifying Deposits, during the 30-Day Evaluation Period will earn 1.20% APY on savings balances ( ... Missing: Hasbank | Show results with:Hasbank. The Ally Bank Savings Account is a high-yield savings option with no minimum deposit required to open and zero monthly fees. At 3.85% APY, this ... Missing: Hasbank | Show results with:Hasbank. The Openbank High Yield Savings has a high savings account interest rate of 5.00% (vary depending on location) APY. It doesn't charge any ... Missing: Hasbank | Show results with:Hasbank. 

'The highest interest rate savings account at Hasbank is 4.80% for MNT and 1.00% for USD, EUR, and other currencies.'

In [20]:
import gradio as gr

def get_agent_response(user_input):
    res = agent.run(user_input)
    res = translate_client.translate(res, target_language="mn")["translatedText"]
    return res

In [21]:
# Create the Gradio interface
iface = gr.Interface(
    fn=get_agent_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs=gr.Textbox(),
    title="Multilingual ReAct Agent",
    description="Ask questions and get responses in Mongolian. The agent remembers the conversation history."
)

# Launch the interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [22]:
iface.close()

Closing server running on port: 7860
